In [ ]:
from ndtools import staged_max_flow as smf

from pathlib import Path
import json

from tsum import tsum
import torch

# Reliability analysis of pressure regularisation plant

## Case 1: Original system

### Load data

In [2]:
DATASET = Path(r"data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))


In [3]:
def s_fun(comps_st):
    flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, nodes, edges, probs_dict, target_flow = 90.0 )

    sys_st = 1 if sys_st_str == 's' else 0
    return flow, sys_st, min_comp_state

row_names = list(probs_dict.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

### Get rules for system event

In [4]:
sys_surv_st = 1 

result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-5,
    output_dir="tsum_res"
) 

---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 0, System value: 55.0. Total samples: 100000.
New rule (No. of conditions: 1): {'x10': ('<=', 0), 'sys': ('<=', 0)}
Updated sys_vals: [55.0]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 0, Failure rules: 1
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 90.0. Total samples: 100000.
New rule (No. of conditions: 16): {'x2': ('>=', 1), 'x3': ('>=', 1), 'x6': ('>=', 1), 'x10': ('>=', 1), 'x12': ('>=', 1), 'x15': ('>=', 1), 'x16': ('>=', 1), 'x17': ('>=', 1), 'x18': ('>=', 1), 'x19': ('>=', 1), 'x22': ('>=', 1), 'x25': ('>=', 1), 'x29': ('>=', 1), 'x33': ('>=', 1), 'x37': ('>=', 1), 'x39': ('>=', 1)

## Case 2: Expanded system

In [5]:
DATASET = Path(r"data") 

nodes = json.loads((DATASET / "expanded_nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "expanded_edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "expanded_probs.json").read_text(encoding="utf-8"))

In [ ]:
def s_fun(comps_st):
    flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, nodes, edges, probs_dict, target_flow = 90.0 )

    sys_st = 1 if sys_st_str == 's' else 0
    return flow, sys_st, min_comp_state

row_names = list(probs_dict.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

### Get rules for system event

In [7]:
sys_surv_st = 1 

result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-5,
    output_dir="tsum_res_expanded"
) 

---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 90.0. Total samples: 100000.
New rule (No. of conditions: 15): {'x3': ('>=', 1), 'x7': ('>=', 1), 'x10': ('>=', 1), 'x12': ('>=', 1), 'x18': ('>=', 1), 'x19': ('>=', 1), 'x22': ('>=', 1), 'x25': ('>=', 1), 'x30': ('>=', 1), 'x34': ('>=', 1), 'x37': ('>=', 1), 'x39': ('>=', 1), 'x44': ('>=', 1), 'x56': ('>=', 1), 'x57': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [90.0]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 1, Failure rules: 0
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 0, System value: 55.0. Total samples: 100000.
New rule (No. of conditions: 1): {'x10': ('<=', 0), 'sys': ('<=', 0